In [1]:
from langchain_openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
llm = OpenAI(temperature=0.9)
name = llm.invoke("I want to open a restaurant for Indian food. Suggest a fancy name for this.")
print(name)



"Maharaja's Palace" or "The Royal Taj"


# Prompt Templates

In [16]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

print(llm.invoke(p))

I want to open a restaurant for Italian food. Suggest a fency name for this.


"La Dolce Vita Trattoria"


# Chains

input --> chain1 --> intermediate output --> chain2 --> final output  

`Simple sequential` chain `does not` return `intermediate output`.  
Whereas `regular sequential` chain `does` return `the intermiadiate output`.

In [11]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.invoke("Mexican")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


{'cuisine': 'Mexican', 'text': '\n\n"Casa de Sabor"'}

##### Before diving deep into Sequential chains, lets copy paste the above code and name it `name_chain` and add another chain- `food_items_chain`

### Simple Sequential Chain

In [ ]:
# Setting up chains

llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)
name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [ ]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.invoke("Indian")
print(content)

{'input': 'Indian', 'output': '\n\n1. Chicken Tikka Masala\n2. Vegetable Samosas\n3. Lamb Vindaloo\n4. Palak Paneer (spinach and cottage cheese curry)\n5. Tandoori Chicken\n6. Chana Masala (chickpea curry)\n7. Aloo Gobi (potato and cauliflower curry)\n8. Butter Chicken\n9. Biryani (spiced rice dish)\n10. Naan bread\n11. Mango Lassi (yogurt drink)\n12. Vegetable Korma\n13. Dal Makhani (creamy lentil dish)\n14. Tandoori Fish\n15. Gulab Jamun (sweet fried dumplings in syrup)'}


### Sequential Chain

In [ ]:
# Setting up chains

llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [19]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

content = chain.invoke({"cuisine": "Indian"})
print(content)

{'cuisine': 'Indian', 'restaurant_name': '\n\n"Spice Palace"', 'menu_items': '\n\n1. Chicken Tikka Masala\n2. Lamb Vindaloo\n3. Vegetable Biryani\n4. Samosas\n5. Garlic Naan\n6. Tandoori Shrimp\n7. Saag Paneer\n8. Chana Masala\n9. Mango Lassi\n10. Butter Chicken\n11. Aloo Gobi\n12. Tandoori Chicken\n13. Palak Paneer\n14. Vegetable Korma\n15. Gulab Jamun (dessert)'}


# Agents

In [12]:
from langchain_openai import OpenAI
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain.chains.llm_math.base import LLMMathChain
from langchain.agents import initialize_agent, AgentType, Tool

llm = OpenAI(temperature=0)

# Search tool
search = DuckDuckGoSearchRun()

# Math chain as a tool
llm_math_chain = LLMMathChain.from_llm(llm=llm)
math_tool = Tool(
    name="Calculator",
    func=llm_math_chain.run,
    description="Useful for when you need to do math"
)

# Agent setup
tools = [search, math_tool]
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Run query
result = agent.invoke({"input":"Find the GDP of the US in 2024 from the web and then add 5 to it. Give the answer in trillions."})
print(result)




> Entering new AgentExecutor chain...
 I should use duckduckgo_search to find the GDP of the US in 2024.
Action: duckduckgo_search
Action Input: "US GDP 2024"

d:\code\ai-learning-projects\nlp-genai\.venv\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: The GDP figure in 2024 was $29,184,900 million, United States is the world's leading economy with regard to GDP, as can be seen in the … Mar 27, 2025 · Real GDP increased 2.8 percent in 2024 (from the 2023 annual level to the 2024 annual level), the same as previously … Jul 5, 2024 · This graph shows a forecast of the gross domestic product of the United States of America for fiscal years 2024 to 2034. GDP … The Gross Domestic Product (GDP) in the United States was worth 29184.89 billion US dollars in 2024, according to official data from the World … United States GDP Is estimated To be $27,614,913,366,428 by the end of 2024 by World Economics. This estimate for United States …
Thought: Now I need to add 5 to the GDP figure.
Action: Calculator
Action Input: 29184.89 billion + 5 trillion
Observation: Answer: 34184890000000.0
Thought: I now know the final answer.
Final Answer: The GDP of the US in 2024 plus 5 trillion is $34.18 trillion.

> Finished chain.
{'input': 'Find t

# Memory

### Conversation Buffer Memory

In [22]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)

name = chain.invoke("Mexican")
print(name)

{'cuisine': 'Mexican', 'history': '', 'text': '\n\n"El Sabor Exquisito" (The Exquisite Flavor)'}


In [23]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabor Exquisito" (The Exquisite Flavor)


### ConversationChain 

In [27]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))

convo.invoke("Who won the first cricket world cup?")

{'input': 'Who won the first cricket world cup?',
 'history': '',
 'response': ' The first cricket world cup was won by the West Indies in 1975. They defeated Australia in the final match by 17 runs. The tournament was held in England and featured eight teams. The West Indies, led by captain Clive Lloyd, had a strong team with players like Viv Richards, Gordon Greenidge, and Andy Roberts. They were considered the favorites going into the tournament and lived up to expectations by winning the final. The West Indies went on to win the next world cup in 1979 as well, solidifying their dominance in the sport.'}

In [28]:
convo.invoke("How much is 5+5?")

{'input': 'How much is 5+5?',
 'history': 'Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was won by the West Indies in 1975. They defeated Australia in the final match by 17 runs. The tournament was held in England and featured eight teams. The West Indies, led by captain Clive Lloyd, had a strong team with players like Viv Richards, Gordon Greenidge, and Andy Roberts. They were considered the favorites going into the tournament and lived up to expectations by winning the final. The West Indies went on to win the next world cup in 1979 as well, solidifying their dominance in the sport.',
 'response': ' The answer to 5+5 is 10. This is a basic addition problem that can be solved by adding 5 to 5, resulting in a sum of 10. Addition is a fundamental mathematical operation that involves combining two or more numbers to find a total or sum. It is often represented with the "+" symbol and is one of the first math concepts taught in elementary school. '}

In [29]:
convo.invoke("Who was the captain ofthe winning team?")

{'input': 'Who was the captain ofthe winning team?',
 'history': 'Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was won by the West Indies in 1975. They defeated Australia in the final match by 17 runs. The tournament was held in England and featured eight teams. The West Indies, led by captain Clive Lloyd, had a strong team with players like Viv Richards, Gordon Greenidge, and Andy Roberts. They were considered the favorites going into the tournament and lived up to expectations by winning the final. The West Indies went on to win the next world cup in 1979 as well, solidifying their dominance in the sport.\nHuman: How much is 5+5?\nAI:  The answer to 5+5 is 10. This is a basic addition problem that can be solved by adding 5 to 5, resulting in a sum of 10. Addition is a fundamental mathematical operation that involves combining two or more numbers to find a total or sum. It is often represented with the "+" symbol and is one of the first math concepts t

In [30]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was won by the West Indies in 1975. They defeated Australia in the final match by 17 runs. The tournament was held in England and featured eight teams. The West Indies, led by captain Clive Lloyd, had a strong team with players like Viv Richards, Gordon Greenidge, and Andy Roberts. They were considered the favorites going into the tournament and lived up to expectations by winning the final. The West Indies went on to win the next world cup in 1979 as well, solidifying their dominance in the sport.
Human: How much is 5+5?
AI:  The answer to 5+5 is 10. This is a basic addition problem that can be solved by adding 5 to 5, resulting in a sum of 10. Addition is a fundamental mathematical operation that involves combining two or more numbers to find a total or sum. It is often represented with the "+" symbol and is one of the first math concepts taught in elementary school. 
Human: Who was the captain ofthe winning

##### See the llm remembers the context but this can be restricted to save memory

# ConversationBufferWindowMemory

In [34]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.invoke("Who won the first cricket world cup?")

{'input': 'Who won the first cricket world cup?',
 'history': '',
 'response': ' The first cricket world cup was won by the West Indies in 1975.'}

In [35]:
convo.invoke("How much is 5+5?")

{'input': 'How much is 5+5?',
 'history': 'Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was won by the West Indies in 1975.',
 'response': '  5+5 is equal to 10.'}

In [36]:
convo.invoke("Who was the captain of the winning team?")

{'input': 'Who was the captain of the winning team?',
 'history': 'Human: How much is 5+5?\nAI:   5+5 is equal to 10.',
 'response': ' I do not have enough context to accurately answer that question. Can you provide more information?'}

##### Now the llm do not have the context, hence cannot answer the question